# RT-DETR Pretraining with SHIFT-Discrete Dataset

## Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = ",".join([f"{i+DEVICE_NUM}" for i in range(0, ADDITIONAL_GPU+1)])
environ["CUDA_VISIBLE_DEVICES"]

## Imports

In [ ]:
from os import path

import torch
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda")  # torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

In [ ]:
# Tqdm Test
for _ in tqdm(range(100)):
    pass

In [ ]:
PROJECT_NAME = "APT_SHIFT_Pretraining"
RUN_NAME = "RT-DETR_R50"

# WandB Initialization
wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Define Dataset

In [ ]:
DATA_ROOT = path.join(".", "data")

dataset = DatasetHolder(
    train=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
    valid=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
    test=SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
)

In [ ]:
dataset.train[1]['front'].keys()

In [ ]:
dataset.train[999]

In [ ]:
dataset.train[1000]['front']['images'].shape  # should be (batch_size, num_channels, height, width)

## DataLoader

In [ ]:
# Set Batch Size
BATCH_SIZE = 4, 46, 46
BATCH_SIZE = 50, 250, 250  # A100 or H100

# Dataset Configs
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

In [ ]:
class DatasetAdapterForTransformers(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        image = item['images'].squeeze(0)

        # Convert to COCO_Detection Format
        annotations = []
        target = dict(image_id=idx, annotations=annotations)
        for box, cls in zip(item['boxes2d'], item['boxes2d_classes']):
            x1, y1, x2, y2 = box.tolist()  # from Pascal VOC format (x1, y1, x2, y2)
            width, height = x2 - x1, y2 - y1
            annotations.append(dict(
                bbox=[x1, y1, width, height],  # to COCO format: [x, y, width, height]
                category_id=cls.item(),
                area=width * height,
                iscrowd=0
            ))

        # Following prepare_coco_detection_annotation's expected format
        # RT-DETR ImageProcessor converts the COCO bbox to center format (cx, cy, w, h) during preprocessing
        # But, eventually re-converts the bbox to Pascal VOC (x1, y1, x2, y2) format after post-processing
        return dict(image=image, target=target)

In [ ]:
def collate_fn(batch, preprocessor=None):
    images = [item['image'] for item in batch]
    if preprocessor is not None:
        target = [item['target'] for item in batch]
        return preprocessor(images=images, annotations=target, return_tensors="pt")
    else:
        # If no preprocessor is provided, just assume images are already in tensor format
        return dict(
            pixel_values=dict(pixel_values=torch.stack(images)),
            labels=[dict(
                class_labels=item['boxes2d_classes'].long(),
                boxes=item["boxes2d"].float()
            ) for item in batch]
        )

## Load Model

In [ ]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig, SwinConfig, ResNetConfig
from transformers.image_utils import AnnotationFormat

In [ ]:
USE_PRETRAINED_MODEL = False
USE_SWIN_T_BACKBONE = False
USE_SHIFT_BACKBONE = False

In [ ]:
reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = NUM_CLASSES

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 800, "width": 800}
reference_preprocessor.do_resize = False

In [ ]:
if USE_PRETRAINED_MODEL:
    # Load the pre-trained model
    model = RTDetrForObjectDetection.from_pretrained(reference_model_id, config=reference_config, torch_dtype=torch.float32, ignore_mismatched_sizes=True)
else:
    # Set the backbone configuration
    if USE_SHIFT_BACKBONE:
        if USE_SWIN_T_BACKBONE:
            backbone_url = "https://github.com/robustaim/ContinualTTA_ObjectDetection/releases/download/backbone_converted/swin_tiny_patch4_window7_shift_from_detectron2.pth"
            reference_config.backbone_config = SwinConfig(
                embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24], output_hidden_states=True,
                window_size=7, out_features=["stage1", "stage2", "stage3"]  # stride 8·16·32
            )
        else:
            backbone_url = "https://github.com/robustaim/ContinualTTA_ObjectDetection/releases/download/backbone_converted/resnet50_shift_from_detectron2.pth"
            reference_config.backbone_config = ResNetConfig(depths=[3,4,6,3], out_indices=(2, 3, 4))
    else:
        reference_config.backbone_config = False
        reference_config.use_pretrained_backbone = True
        if USE_SWIN_T_BACKBONE:
            reference_config.backbone = "microsoft/swin-tiny-patch4-window7-224"
        else:
            reference_config.backbone = "microsoft/resnet-50"

    # Initialize a new model with the reference configuration
    model = RTDetrForObjectDetection(config=reference_config)
    if USE_SHIFT_BACKBONE:
        backbone_state = torch.hub.load_state_dict_from_url(backbone_url, map_location="cpu")
        model.model.backbone.model.load_state_dict(backbone_state, strict=False)
    elif USE_SWIN_T_BACKBONE:
        model.model.backbone.model.forward.__kwdefaults__['interpolate_pos_encoding'] = True
model.to(device)

In [ ]:
test_d = DatasetAdapterForTransformers(dataset.train)[5]
test_d

In [ ]:
reference_preprocessor(images=test_d['image'], annotations=test_d['target'])

In [ ]:
# Set Epoch Count & Learning Rate
EPOCHS = 100
REAL_BATCH = 150
LEARNING_RATE = 5e-4

training_args = TrainingArguments(
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.05,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE[0],
    per_device_eval_batch_size=BATCH_SIZE[1],
    gradient_accumulation_steps=REAL_BATCH//BATCH_SIZE[0],
    eval_accumulation_steps=BATCH_SIZE[1],
    batch_eval_metrics=True,
    remove_unused_columns=False,
    optim="adamw_torch",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    #eval_steps=500,
    #save_steps=500,
    save_total_limit=20,
    load_best_model_at_end=True,
    metric_for_best_model="mAP@0.50:0.95",
    greater_is_better=True,
    #metric_for_best_model="eval_loss",
    #greater_is_better=False,
    report_to="wandb",
    output_dir="./results/"+RUN_NAME,
    logging_dir="./logs/"+RUN_NAME,
    run_name=RUN_NAME,
    bf16=True,
)

testing_args = TrainingArguments(
    per_device_eval_batch_size=BATCH_SIZE[2],
    batch_eval_metrics=True,
    remove_unused_columns=False,
)

In [ ]:
from transformers.trainer_utils import EvalPrediction
from torchvision.ops import box_convert
from dataclasses import dataclass


@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


def de_normalize_boxes(boxes, height, width):
    # 1. cxcywh → xyxy
    boxes_xyxy_norm = box_convert(boxes, 'cxcywh', 'xyxy')

    # 2. de-normalize (convert to actual pixel coordinates)
    boxes_xyxy_norm[:, [0, 2]] *= width
    boxes_xyxy_norm[:, [1, 3]] *= height
    return boxes_xyxy_norm


def map_compute_metrics(preprocessor=reference_preprocessor, threshold=0.0):
    map_metric = MeanAveragePrecision()
    post_process = preprocessor.post_process_object_detection

    def calc(eval_pred: EvalPrediction, compute_result=False):
        nonlocal map_metric

        if compute_result:
            m_ap = map_metric.compute()
            map_metric.reset()

            per_class_map = {
                f"{CLASSES[idx]}_mAP@0.50:0.95": m_ap.ap_per_class[idx].mean()
                for idx, idx in enumerate(m_ap.matched_classes)
            }

            return {
                "mAP@0.50:0.95": m_ap.map50_95,
                "mAP@0.50": m_ap.map50,
                "mAP@0.75": m_ap.map75,
                **per_class_map
            }
        else:
            preds = ModelOutput(*eval_pred.predictions[1:3])
            labels = eval_pred.label_ids
            sizes = [label['orig_size'].cpu().tolist() for label in labels]

            results = post_process(preds, target_sizes=sizes, threshold=threshold)
            predictions = [Detections.from_transformers(result) for result in results]
            targets = [Detections(
                xyxy=de_normalize_boxes(label['boxes'], *label['orig_size']).cpu().numpy(),
                class_id=label['class_labels'].cpu().numpy(),
            ) for label in labels]

            map_metric.update(predictions=predictions, targets=targets)
            return {}
    return calc, map_metric

In [ ]:
from functools import partial

compute_metrics, compute_results = map_compute_metrics(preprocessor=reference_preprocessor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=DatasetAdapterForTransformers(dataset.train),
    eval_dataset=DatasetAdapterForTransformers(dataset.valid),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=DatasetAdapterForTransformers(dataset.test),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics
)

## Train

In [ ]:
def start_train():
    accelerator = Accelerator()
    try:
        print("INFO: Trying to resume from previous checkpoint")
        compute_results.reset()
        trainer.train(resume_from_checkpoint=True)
    except Exception as e:
        if "No valid checkpoint found" in str(e):
            print(f"ERROR: Failed to resume from checkpoint - {e}")
            print("INFO: Starting training from scratch")
            compute_results.reset()
            trainer.train(resume_from_checkpoint=False)

In [ ]:
if ADDITIONAL_GPU:
    notebook_launcher(start_train, args=(), num_processes=ADDITIONAL_GPU)
else:
    start_train()

In [ ]:
compute_results.compute().plot()

## Evaluate

### Auto Evaluation

In [ ]:
trainer.evaluate()

In [ ]:
tester.evaluate()

### Manual Evaluation

In [ ]:
checkpoint = 31100

In [ ]:
try:
    model = RTDetrForObjectDetection.from_pretrained(f"{training_args.output_dir}/checkpoint-{checkpoint}/", torch_dtype=torch.float32, return_dict=True, local_files_only=True)
    model.to(device)
except Exception:
    pass

In [ ]:
class LabelDataset(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        return item['boxes2d'], item['boxes2d_classes']

In [ ]:
def naive_collate_fn(batch):
    return batch

In [ ]:
targets = []
predictions = []
batch_size = 32

raw_data = DataLoader(LabelDataset(dataset.valid), batch_size=batch_size, collate_fn=naive_collate_fn)
loader = DataLoader(DatasetAdapterForTransformers(dataset.valid), batch_size=batch_size, collate_fn=partial(collate_fn, preprocessor=reference_preprocessor))
for idx, lables, inputs in zip(tqdm(range(len(raw_data))), raw_data, loader):
    sizes = [label['orig_size'].cpu().tolist() for label in inputs['labels']]

    with torch.no_grad():
        outputs = model(pixel_values=inputs['pixel_values'].to(device))

    results = reference_preprocessor.post_process_object_detection(
        outputs, target_sizes=sizes, threshold=0.3
    )

    detections = [Detections.from_transformers(results[i]) for i in range(batch_size)]
    annotations = [Detections(
        xyxy=lables[i][0].cpu().numpy(),
        class_id=lables[i][1].cpu().numpy(),
    ) for i in range(batch_size)]

    targets.extend(annotations)
    predictions.extend(detections)

In [ ]:
len(predictions) == len(targets), len(predictions), len(targets)

In [ ]:
mean_average_precision = MeanAveragePrecision().update(
    predictions=predictions,
    targets=targets,
).compute()
per_class_map = {
    f"{CLASSES[idx]}_mAP@0.95": mean_average_precision.ap_per_class[idx].mean()
    for idx, idx in enumerate(mean_average_precision.matched_classes)
}

print(f"mAP@0.95: {mean_average_precision.map50_95:.2f}")
print(f"map50: {mean_average_precision.map50:.2f}")
print(f"map75: {mean_average_precision.map75:.2f}")
for key, value in per_class_map.items():
    print(f"{key}: {value:.2f}")